In [ ]:
from googleapiclient.discovery import build
import pandas as pd
import isodate
from datetime import datetime
from dotenv import dotenv_values
from google.cloud import bigquery
from google.oauth2 import service_account
import os

In [48]:
config = dotenv_values()
api_key = config['api_key']

In [ ]:
youtube = build("youtube", "v3", developerKey=api_key)


region_codes = [
    'US', 'CA', 'GB', 'AU', 'IN', 'JP', 'KR', 'BR', 'MX', 'FR', 'DE', 'RU', 'IT', 'ES',
    'AR', 'CO', 'CL', 'NL', 'TR', 'SA', 'AE', 'EG', 'ID', 'MY', 'TH', 'VN', 'SG', 'NG',
    'KE', 'ZA', 'PK', 'BD', 'UA', 'PL', 'SE', 'CH', 'BE', 'NO', 'DK', 'FI', 'IE', 'NZ',
    'PH', 'HK', 'TW', 'IL', 'RO', 'HU', 'CZ', 'GR', 'PT', 'SK', 'AT'
]


video_data = []

for region in region_codes:
    response = youtube.videos().list(
        part="snippet,contentDetails,statistics",
        chart="mostPopular",
        regionCode=region,
        maxResults=50
    ).execute()

    for item in response['items']:
        snippet = item['snippet']
        stats = item.get('statistics', {})
        content = item['contentDetails']
        
        video_data.append({
            'video_id': item['id'],
            'title': snippet.get('title'),
            'description': snippet.get('description'),
            'channel_title': snippet.get('channelTitle'),
            'published_at': datetime.strptime(snippet.get('publishedAt'), "%Y-%m-%dT%H:%M:%SZ"),
            'category_id' : snippet.get('categoryId'),
            'default_language' : snippet.get('defaultLanguage'),
            'tags': ', '.join(snippet.get('tags', [])),
            'duration_seconds': isodate.parse_duration(content['duration']).total_seconds(),
            'view_count': int(stats.get('viewCount', 0)),
            'like_count': int(stats.get('likeCount', 0)),
            'comment_count': int(stats.get('commentCount', 0)),
        })




unique_data = {video['video_id']: video for video in video_data}
video_data = list(unique_data.values())



load_date = datetime.utcnow()
for video in video_data:
    video["load_date"] = load_date


df = pd.DataFrame(video_data)



C:\Users\tanju\AppData\Local\Temp\ipykernel_2348\1652838759.py:51: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  load_date = datetime.utcnow()


In [50]:
df.head(3)

,video_id,title,description,channel_title,published_at,category_id,default_language,tags,duration_seconds,view_count,like_count,comment_count,load_date
0,wSa-sjaLVOM,Five Nights at Freddy's: The Secret of the Mimic,Five Nights at Freddy's is BACK with a game th...,Markiplier,2025-06-17 00:09:51,20,en,"markiplier, five nights at freddy's, fnaf, sec...",7456.0,2049602,143796,4521,2025-06-17 21:38:40.707027
1,URlPXepBZdo,Clipse - So Be It (Official Music Video),New Album 'Let God Sort Em Out' July 11\nPre-O...,clipseVEVO,2025-06-17 16:00:46,10,None,"Clipse, Roc Nation Distribution, Hip Hop/Rap, ...",199.0,323100,55710,6661,2025-06-17 21:38:40.707027
2,uLguU7WLreA,The Naked Gun | Official Trailer (2025 Movie) ...,Justice has a new Daddy. Watch the new trailer...,Paramount Pictures,2025-06-16 13:00:13,1,None,"The Naked Gun, Liam Neeson, Pamela Anderson, N...",150.0,5977070,50677,5473,2025-06-17 21:38:40.707027


In [51]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1478 entries, 0 to 1477
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   video_id          1478 non-null   object        
 1   title             1478 non-null   object        
 2   description       1478 non-null   object        
 3   channel_title     1478 non-null   object        
 4   published_at      1478 non-null   datetime64[ns]
 5   category_id       1478 non-null   object        
 6   default_language  525 non-null    object        
 7   tags              1478 non-null   object        
 8   duration_seconds  1478 non-null   float64       
 9   view_count        1478 non-null   int64         
 10  like_count        1478 non-null   int64         
 11  comment_count     1478 non-null   int64         
 12  load_date         1478 non-null   datetime64[ns]
dtypes: datetime64[ns](2), float64(1), int64(3), object(7)
memory usage: 150.2+ KB


In [52]:
df.columns

Index(['video_id', 'title', 'description', 'channel_title', 'published_at',
       'category_id', 'default_language', 'tags', 'duration_seconds',
       'view_count', 'like_count', 'comment_count', 'load_date'],
      dtype='object')

In [53]:
df.describe()

,published_at,duration_seconds,view_count,like_count,comment_count,load_date
count,1478,1478.000000,1.478000e+03,1.478000e+03,1478.000000,1478
mean,2025-06-11 21:27:15.525710336,1569.903248,5.875312e+06,1.485989e+05,2578.801083,2025-06-17 21:38:40.707026944
min,2025-05-15 09:05:49,10.000000,2.074000e+04,0.000000e+00,0.000000,2025-06-17 21:38:40.707027
25%,2025-06-09 16:41:50.500000,49.000000,2.769080e+05,7.361750e+03,214.000000,2025-06-17 21:38:40.707026944
50%,2025-06-13 13:27:08.500000,213.000000,7.621575e+05,2.177050e+04,673.000000,2025-06-17 21:38:40.707026944
75%,2025-06-15 17:00:12.249999872,1245.750000,3.484478e+06,9.291450e+04,1891.000000,2025-06-17 21:38:40.707026944
max,2025-06-17 18:55:41,42097.000000,2.517564e+08,5.488167e+06,185025.000000,2025-06-17 21:38:40.707027
std,NaN,4305.538644,1.644656e+07,3.991892e+05,9570.075827,NaN


In [57]:
credentials = service_account.Credentials.from_service_account_file(
    r"C:\Users\tanju\Desktop\upheld-momentum-463013-v7-a9926786a277.json"
)

client = bigquery.Client(credentials=credentials, project='upheld-momentum-463013-v7')



# Set your dataset and table
dataset_id = 'dbt_tdereli'  # e.g. 'dbt_tdereli'
table_id = 'youtube_trending_videos'

table_ref = client.dataset(dataset_id).table(table_id)

# Define table schema (optional but recommended)
schema = [
    bigquery.SchemaField("load_date", "TIMESTAMP"),
    bigquery.SchemaField("video_id", "STRING"),
    bigquery.SchemaField("title", "STRING"),
    bigquery.SchemaField("description", "STRING"),
    bigquery.SchemaField("channel_title", "STRING"),
    bigquery.SchemaField("published_at", "TIMESTAMP"),
    bigquery.SchemaField("category_id", "STRING"),
    bigquery.SchemaField("default_language", "STRING"),
    bigquery.SchemaField("tags", "STRING"),
    bigquery.SchemaField("duration_seconds", "FLOAT"),
    bigquery.SchemaField("view_count", "INTEGER"),
    bigquery.SchemaField("like_count", "INTEGER"),
    bigquery.SchemaField("comment_count", "INTEGER"),
]

# Configure job to overwrite table if exists
job_config = bigquery.LoadJobConfig(
    schema=schema,
    write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE,
)

# Load data into BigQuery
job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
job.result()  # Wait for completion

print(f"Loaded {job.output_rows} rows into {dataset_id}.{table_id}")

c:\Users\tanju\miniconda3\Lib\site-packages\google\cloud\bigquery\_pandas_helpers.py:489: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


Loaded 1478 rows into dbt_tdereli.youtube_trending_videos
